<a href="https://colab.research.google.com/github/kvsmadhulika/Madhulika_koduru/blob/main/Spark_NLP_Venkata_Sai_Madhulika_Koduru.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# This is to setup PySpark and Spark NLP on Colab
!wget http://setup.johnsnowlabs.com/colab.sh -O - | bash

--2024-04-10 20:56:25--  http://setup.johnsnowlabs.com/colab.sh
Resolving setup.johnsnowlabs.com (setup.johnsnowlabs.com)... 51.158.130.125
Connecting to setup.johnsnowlabs.com (setup.johnsnowlabs.com)|51.158.130.125|:80... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/scripts/colab_setup.sh [following]
--2024-04-10 20:56:25--  https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/scripts/colab_setup.sh
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1191 (1.2K) [text/plain]
Saving to: ‘STDOUT’

-                     0%[                    ]       0  --.-KB/s               Installing PySpark 3.2.3 and Spark NLP 5.3.3
setup Colab for PySpark 3.2.3 an

In [ ]:
import sparknlp
spark = sparknlp.start()

print("Spark NLP version: {}".format(sparknlp.version()))
print("Apache Spark version: {}".format(spark.version))

Spark NLP version: 5.3.3
Apache Spark version: 3.2.3


In [ ]:
reviews_df = spark.read.options(mode='FAILFAST',
                               header= True, multiline = True, inferSchema=True, escape='"').csv('/content/drive/MyDrive/reviews.csv')

In [ ]:
reviews_df.dtypes

[('listing_id', 'bigint'),
 ('id', 'bigint'),
 ('date', 'string'),
 ('reviewer_id', 'int'),
 ('reviewer_name', 'string'),
 ('comments', 'string')]

In [ ]:
### Split Dataset
(extra_df,use_df) = reviews_df.randomSplit((0.75,0.25),seed=42)

In [ ]:
from pyspark.sql import functions as F

text_col = 'comments'
Reviews_text = use_df.select("listing_id","id",text_col).filter(F.col(text_col).isNotNull())

In [ ]:
#Screenshot 1
Reviews_text.head(5)

[Row(listing_id=90676, id=972073, comments='Audra has a very beautiful home in a convenient location right between downtown and OSU. Clean, comfortable room was visited often by two very friendly and adorable dogs. I really felt welcomed and at home.'),
 Row(listing_id=90676, id=2527604, comments='Very beatiful home. Great location near High Street, a very nice place with many bars, restaurants and shopping. Very good hosts too, very kind and respectful. I felt very comfortable. '),
 Row(listing_id=90676, id=2999152, comments="Always great to stay at Audra's and Lacey's.  It is not often that I am able to say this, but it truely feels like a home away from home.  I look forward to stayong there again"),
 Row(listing_id=90676, id=3685780, comments="This was my second time staying at Audra's house, and once again I couldn't have asked for a better experience."),
 Row(listing_id=90676, id=8176298, comments='Great home, great location ! Very clean and phenomenal  loft suite and shower.')]

In [ ]:
#NLP Pipeline Construction
from sparknlp.base import DocumentAssembler
documentAssembler = DocumentAssembler().setInputCol(text_col).setOutputCol('document')

In [ ]:
#Tokenizer
from sparknlp.annotator import Tokenizer
tokenizer = Tokenizer().setInputCols(['document']).setOutputCol('tokenized')

In [ ]:
#Normalizer
from sparknlp.annotator import Normalizer
normalizer = Normalizer().setInputCols(['tokenized']).setOutputCol('normalized').setLowercase(True)

In [ ]:
#Lemmatizer
from sparknlp.annotator import LemmatizerModel
lemmatizer = LemmatizerModel.pretrained().setInputCols(['normalized']).setOutputCol('lemmatized')

lemma_antbnc download started this may take some time.
Approximate size to download 907.6 KB
[OK!]


In [ ]:
#Stop Words
# Import the required modules and classes
from sparknlp.base import DocumentAssembler, Pipeline
from sparknlp.annotator import (
    SentenceDetector,
    Tokenizer,
    StopWordsCleaner)

# Step 3: Performs stopwords cleaning
stop_words = (
    StopWordsCleaner().pretrained("stopwords_iso","en")
    .setInputCols(["lemmatized"])
    .setOutputCol("unigrams")
)

stopwords_iso download started this may take some time.
Approximate size to download 2.1 KB
[OK!]


In [ ]:
from sparknlp.annotator import StopWordsCleaner
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords
eng_stopwords = stopwords.words('english')
stopwords_cleaner = StopWordsCleaner().setInputCols(['lemmatized']).setOutputCol('unigrams').setStopWords(eng_stopwords)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
#ngram generator
from sparknlp.annotator import NGramGenerator

ngrammer = NGramGenerator() \
    .setInputCols(['lemmatized']) \
    .setOutputCol('ngrams') \
    .setN(3) \
    .setEnableCumulative(True) \
    .setDelimiter('_')

In [ ]:
from sparknlp.annotator import PerceptronModel
pos_tagger = PerceptronModel.pretrained('pos_anc').setInputCols(['document', 'lemmatized']).setOutputCol('pos')

pos_anc download started this may take some time.
Approximate size to download 3.9 MB
[OK!]


In [ ]:
from sparknlp.annotator import Chunker
allowed_tags = ['<JJ>+<NN>', '<NN>+<NN>']
chunker = Chunker().setInputCols(['document', 'pos']).setOutputCol('ngrams').setRegexParsers(allowed_tags)

In [ ]:
#Finisher
from sparknlp.base import Finisher
finisher = Finisher().setInputCols(['unigrams','ngrams'])

In [ ]:
from pyspark.ml import Pipeline
pipeline1 = Pipeline().setStages([documentAssembler,
                 tokenizer,
                 normalizer,
                 lemmatizer,
                 stopwords_cleaner,
                 pos_tagger,
                 chunker,
                 finisher])

In [ ]:
from pyspark.ml import Pipeline
pipeline = Pipeline().setStages([documentAssembler,
                 tokenizer,
                 normalizer,
                 lemmatizer,
                 stop_words,
                 pos_tagger,
                 chunker,
                 finisher])

In [ ]:
Processed_review = pipeline.fit(Reviews_text).transform(Reviews_text)

In [ ]:
Processed_review.limit(5).show()

+----------+-------+--------------------+--------------------+--------------------+
|listing_id|     id|            comments|   finished_unigrams|     finished_ngrams|
+----------+-------+--------------------+--------------------+--------------------+
|     90676| 972073|Audra has a very ...|[audra, beautiful...|[beautiful home, ...|
|     90676|2527604|Very beatiful hom...|[beatiful, home, ...|[beatiful home, G...|
|     90676|2999152|Always great to s...|[great, stay, aud...|                  []|
|     90676|3685780|This was my secon...|[second, time, st...|[second time, tim...|
|     90676|8176298|Great home, great...|[great, home, gre...|[Great home, grea...|
+----------+-------+--------------------+--------------------+--------------------+



In [ ]:
from pyspark.sql.functions import concat
Processed_review = Processed_review.withColumn('final',
     concat(F.col('finished_unigrams'),
            F.col('finished_ngrams')))


In [ ]:
#Screenshot 2
Processed_review.select('final').limit(5).show(truncate=90)

+------------------------------------------------------------------------------------------+
|                                                                                     final|
+------------------------------------------------------------------------------------------+
|[audra, beautiful, home, convenient, location, right, downtown, osu, clean, comfortable...|
|[beatiful, home, great, location, near, high, street, nice, place, bar, restaurant, sho...|
|[great, stay, audras, laceys, able, truely, feel, like, home, away, home, look, forward...|
|  [second, time, stay, audras, house, couldnt, ask, experience, second time, time staying]|
|[great, home, great, location, clean, phenomenal, loft, suite, shower, Great home, grea...|
+------------------------------------------------------------------------------------------+



In [ ]:
from pyspark.ml.feature import CountVectorizer

tfizer = CountVectorizer(inputCol='final', outputCol='tf_features')
tf_model = tfizer.fit(Processed_review)
tf_result = tf_model.transform(Processed_review)

In [ ]:
from pyspark.ml.feature import IDF

idfizer = IDF(inputCol='tf_features', outputCol='tf_idf_features')
idf_model = idfizer.fit(tf_result)
tfidf_result = idf_model.transform(tf_result)

In [ ]:
from pyspark.ml.clustering import LDA

num_topics = 8
max_iter = 10

lda = LDA(k=num_topics, maxIter=max_iter, featuresCol='tf_idf_features')
lda_model = lda.fit(tfidf_result)

In [ ]:
from pyspark.sql import types as T

vocab = tf_model.vocabulary

def get_words(token_list):
     return [vocab[token_id] for token_id in token_list]

udf_to_words = F.udf(get_words, T.ArrayType(T.StringType()))

In [ ]:
#Screenshot 3
from os import truncate
top_words = 10

topics = lda_model.describeTopics(top_words).withColumn('topicWords', udf_to_words(F.col('termIndices')))
topics.select('topic', 'topicWords').show(truncate=False)

/usr/local/lib/python3.10/dist-packages/pyspark/sql/context.py:125: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


+-----+----------------------------------------------------------------------------------------------------+
|topic|topicWords                                                                                          |
+-----+----------------------------------------------------------------------------------------------------+
|0    |[german, village, German Village, nice, nice place, park, location, great, night, stay]             |
|1    |[great, stay, place, house, clean, location, home, host, comfortable, need]                         |
|2    |[wait, Great hosts, brent, lucy, good communication, e, adam, alot, outdoor space, cant]            |
|3    |[time, stay, place, columbus, host, great, love, location, amazing, home]                           |
|4    |[Good place, I’ve stayed, good, place, nice, house, quick stay, stay, clean, location]              |
|5    |[Great space, good place, stay, house, place, great, space, pack, host, columbus]                   |
|6    |[fun, value,

In [ ]:
# Shows the result
transformed = lda_model.transform(tfidf_result)
#transformed.show(20)
transformed.filter(transformed.listing_id=='90676').show(10)

+----------+--------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|listing_id|      id|            comments|   finished_unigrams|     finished_ngrams|               final|         tf_features|     tf_idf_features|   topicDistribution|
+----------+--------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|     90676|  972073|Audra has a very ...|[audra, beautiful...|[beautiful home, ...|[audra, beautiful...|(68748,[3,4,8,9,2...|(68748,[3,4,8,9,2...|[0.00129703022913...|
|     90676| 2527604|Very beatiful hom...|[beatiful, home, ...|[beatiful home, G...|[beatiful, home, ...|(68748,[1,2,4,5,7...|(68748,[1,2,4,5,7...|[0.00114311816851...|
|     90676| 2999152|Always great to s...|[great, stay, aud...|                  []|[great, stay, aud...|(68748,[0,1,8,30,...|(68748,[0,1,8,30,...|[0.00186

In [ ]:
transformed.dtypes

[('listing_id', 'bigint'),
 ('id', 'bigint'),
 ('comments', 'string'),
 ('finished_unigrams', 'array<string>'),
 ('finished_ngrams', 'array<string>'),
 ('final', 'array<string>'),
 ('tf_features', 'vector'),
 ('tf_idf_features', 'vector'),
 ('topicDistribution', 'vector')]

In [ ]:
from pyspark.ml.feature import VectorAssembler

docTopDF = transformed.select("listing_id","topicDistribution")

def extract(row):
    return (row.listing_id,) + tuple(row.topicDistribution.toArray().tolist())

documentTopics_df = docTopDF.rdd.map(extract).toDF(["listing_id", "topic1","topic2","topic3","topic4", "topic5","topic6", "topic7","topic8"])
documentTopics_df.show()


+----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|listing_id|              topic1|              topic2|              topic3|              topic4|              topic5|              topic6|              topic7|              topic8|
+----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|     90676|0.001297030229133...|  0.8626817077479316| 0.00122915270835834|0.001396244563700...| 0.12959703558134322|0.001268489270009...|0.001290981106079...|0.001239358793444...|
|     90676|0.001143118168519...|  0.9920903868861105|0.001082775912342...|0.001229993151383...|0.001107045430203291|0.001117416770433...|0.001137448102311...|0.001091815578696...|
|     90676|0.001861325816372...|  0.5096088102862382|0.001764503839357...|0.002004234102623794

In [ ]:
averages_df = documentTopics_df.groupBy("listing_id").avg("topic1", "topic2", "topic3","topic4","topic5","topic6")
averages_df.show(12)

+------------------+--------------------+-------------------+--------------------+--------------------+--------------------+--------------------+
|        listing_id|         avg(topic1)|        avg(topic2)|         avg(topic3)|         avg(topic4)|         avg(topic5)|         avg(topic6)|
+------------------+--------------------+-------------------+--------------------+--------------------+--------------------+--------------------+
|          23004939|0.024466548502732326|0.47168290218155234|0.013751583876648564|  0.2039260057313122| 0.04368558277428327| 0.05566055349176148|
|          42693464| 0.04233351122135541| 0.5323187598625442| 0.03474041203597453| 0.17773814931910276| 0.06915349712287708| 0.07435588464352186|
|          44121484| 0.10817767145508474| 0.5581778046753376| 0.03841484803648517|  0.1311589787430937| 0.02593164555313422|0.027897548580263738|
|          47746544| 0.19409073348271225| 0.5189216210037323|0.002537050783186...| 0.10672715017004683| 0.03529285985433474|

In [ ]:
averages_df.filter(averages_df.listing_id=='1217678').show()

+----------+--------------------+------------------+-------------------+-------------------+-------------------+-------------------+
|listing_id|         avg(topic1)|       avg(topic2)|        avg(topic3)|        avg(topic4)|        avg(topic5)|        avg(topic6)|
+----------+--------------------+------------------+-------------------+-------------------+-------------------+-------------------+
|   1217678|0.041437341045037275|0.5844896691088052|0.05341341072744864|0.18231522386666116|0.04423002227042448|0.03695629136387339|
+----------+--------------------+------------------+-------------------+-------------------+-------------------+-------------------+



In [ ]:
listings_df =spark.read.options(mode='FAILFAST',
                              header = True,
                              multiLine=True,
                              inferSchema=True,
                              escape='"').csv("/content/drive/MyDrive/listings.csv")

In [ ]:
listings_df = listings_df.select("id","host_name","host_is_superhost","number_of_reviews", "review_scores_rating", "price")


In [ ]:
totals_df = documentTopics_df.join(listings_df,documentTopics_df.listing_id == listings_df.id).drop(listings_df.id)
totals_df.show(truncate=True)

+----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------+-----------------+-----------------+--------------------+-------+
|listing_id|              topic1|              topic2|              topic3|              topic4|              topic5|              topic6|              topic7|              topic8|    host_name|host_is_superhost|number_of_reviews|review_scores_rating|  price|
+----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------+-----------------+-----------------+--------------------+-------+
|     90676|0.001297030229133...|  0.8626817077479316| 0.00122915270835834|0.001396244563700...| 0.12959703558134322|0.001268489270009...|0.001290981106079...|0.001239358793444...|Audra & Lacey|                t|        

In [ ]:
#Screenshot 4
from pyspark.ml.feature import StringIndexer
totals_df = StringIndexer(
    inputCol='host_is_superhost',
    outputCol='superhost',
    handleInvalid='keep').fit(totals_df).transform(totals_df)

totals_df.show(5, truncate=True)

+----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------+-----------------+-----------------+--------------------+-------+---------+
|listing_id|              topic1|              topic2|              topic3|              topic4|              topic5|              topic6|              topic7|              topic8|    host_name|host_is_superhost|number_of_reviews|review_scores_rating|  price|superhost|
+----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------+-----------------+-----------------+--------------------+-------+---------+
|     90676|0.001297030229133...|  0.8626817077479316| 0.00122915270835834|0.001396244563700...| 0.12959703558134322|0.001268489270009...|0.001290981106079...|0.001239358793444...|Audra & La

In [ ]:
required_features = ['topic1','topic2','topic3','topic4','topic5','topic6']
assembler = VectorAssembler(inputCols=required_features, outputCol='features')
transformed_data = assembler.transform(totals_df)

In [ ]:
(training_data, test_data) = transformed_data.randomSplit([0.8,0.2])

In [ ]:
from pyspark.ml.classification import RandomForestClassifier
rf = RandomForestClassifier(labelCol='superhost',
                            featuresCol='features',
                            maxDepth=5)

In [ ]:
model = rf.fit(training_data)

In [ ]:
predictions = model.transform(test_data)

In [ ]:
# Evaluate our model
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
evaluator = MulticlassClassificationEvaluator(
    labelCol='superhost',
    predictionCol='prediction',
    metricName='accuracy')

In [ ]:
#Screenshot 5
accuracy = evaluator.evaluate(predictions)
print('Test Accuracy = ', accuracy)

Test Accuracy =  0.71357765094623


In [ ]:
#Screenshot 6
from sparknlp.pretrained import PretrainedPipeline
pipeline = PretrainedPipeline("detect_language_21", lang = "xx")

pipeline.annotate('Audra has a very beautiful home in a convenient location right between downtown and OSU. Clean, comfortable room was visited often by two very friendly and adorable dogs. I really felt welcomed and at home.')

detect_language_21 download started this may take some time.
Approx size to download 7.7 MB
[OK!]


{'document': ['Audra has a very beautiful home in a convenient location right between downtown and OSU. Clean, comfortable room was visited often by two very friendly and adorable dogs. I really felt welcomed and at home.'],
 'sentence': ['Audra has a very beautiful home in a convenient location right between downtown and OSU.',
  'Clean, comfortable room was visited often by two very friendly and adorable dogs.',
  'I really felt welcomed and at home.'],
 'language': ['en']}